## Differential Drive Robot Position Controller (Turtlebot 3)
#### Sebastián Palacio Betancur

#### Diferential drive robot kinematics
Taken from: https://aleksandarhaber.com/tutorial-on-simple-position-controller-for-differential-drive-robot-with-simulation-and-animation-in-python/


<img src="images/DDR.png" width="30%"/>

The kinematics system of differential equations describing the robot motion is:
$$
    \begin{equation}
        \begin{array}{l}
        \dot{x} =\dfrac{r\dot{\phi}_{L}}{2}\cos(\theta) +\dfrac{r\dot{\phi}_{R}}{2}\cos(\theta) \\[0.3cm]
        \dot{y} =\dfrac{r\dot{\phi}_{L}}{2}\sin(\theta) +\dfrac{r\dot{\phi}_{R}}{2}\sin(\theta)  \\[0.3cm]
        \dot{\theta}   = -\dfrac{r}{s}\dot{\phi}_{L}+ \dfrac{r}{s}\dot{\phi}_{R}
        \end{array}
    \end{equation}
$$

And the relation between $v_{B}$, $\dot{\theta}$ and $\dot{\phi}_{L}$, $\dot{\phi}_{R}$ is given by:

$$    
    \begin{equation}  
        \begin{bmatrix}
            v_{B} \\
            \dot{\theta}  
        \end{bmatrix} =
        \begin{bmatrix}
            \frac{r}{2} & \frac{r}{2} \\
            -\frac{r}{s} & \frac{r}{s}
        \end{bmatrix}
        \begin{bmatrix}
            \dot{\phi}_{L}  \\
            \dot{\phi}_{R}
        \end{bmatrix}
    \end{equation}
$$
#### Desired control
Let $x_{D}$  and $y_{D}$ be the $x$ and $y$ coordinates of the desired (target or reference) location of the robot center $B$. Determine a sequence of wheel angular velocity control variables $\dot{\phi}_{L}$ and $\dot{\phi}_{R}$ such that the current robot position $(x(t),y(t))$ is equal to the desired robot position $(x_{D},y_{D})$. Here, we assume that at any time instant, we can precisely measure the robot position $(x(t),y(t))$ and the robot orientation angle $\theta(t)$.

<img src="images/DDR_Control.png" width="50%"/>

#### Control diagram:

<img src="images/Control_Diagram.png" width="50%"/>


## Code implementation

This code allows the user to enter a desired number of points (x,y) and see the path followed biy the robot to reach this points in an interactive plot.

In [17]:
pip install simple-pid

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from simple_pid import PID

In [19]:
#Model parameters
# taken from: turtlebot3/turtlebot3_description/urdf/turtlebot3_burger.gazebo.xacro
# <wheelSeparation>0.160</wheelSeparation>
# <wheelDiameter>0.066</wheelDiameter>

r = 0.066 # Radius of the wheels
s = 0.160  # Distance between wheels

In [20]:
# Define the transformation matrix and its inverse
transform_matrix = np.array([[r/2, r/2], [-r/s, r/s]])
inv_transform_matrix = np.linalg.inv(transform_matrix)

# Control parameters
kp_vel = -0.1
ki_vel = 0
kd_vel = 0

kp_angular_vel = 0.3
ki_angular_vel = 0
kd_angular_vel = 0

# Transformation function from v_B, theta_dot to wheel velocities
def calculate_wheel_velocities(v_B, theta_dot):
    v_theta = np.array([v_B, theta_dot])
    phi_dot = np.dot(inv_transform_matrix, v_theta)
    return phi_dot[0], phi_dot[1]

def model(pos, t, x_goals, y_goals):
    x, y, theta = pos
    x_error = x_goals[0] - x
    y_error = y_goals[0] - y

    # PID controllers
    pid_vel = PID(kp_vel, ki_vel, kd_vel, setpoint=0)
    pid_vel.set_auto_mode(enabled=True)
    pid_angular_vel = PID(kp_angular_vel, ki_angular_vel, kd_angular_vel, setpoint=0)
    pid_angular_vel.set_auto_mode(enabled=True)

    theta_goal = np.arctan2(y_error, x_error)

    # Implementing additional logic
    if theta_goal < -np.pi/4 or theta_goal > np.pi/4:
        if y_goals[0] < 0 and y < y_goals[0]:
            theta_goal = -2*np.pi + theta_goal
        elif y_goals[0] >= 0 and y > y_goals[0]:
            theta_goal = 2*np.pi + theta_goal

    # Handle theta wrapping around
    if theta > np.pi - 0.1 and theta_goal <= 0:
        theta_goal = 2*np.pi + theta_goal
    elif theta < -np.pi + 0.1 and theta_goal > 0:
        theta_goal = -2*np.pi + theta_goal

    pid_angular_vel.setpoint = theta_goal
    theta_dot = pid_angular_vel(theta)

    euc_distance = np.sqrt((x_error)**2 + (y_error)**2)
    v_B = pid_vel(euc_distance)

    # Model
    phi_L_dot, phi_R_dot = calculate_wheel_velocities(v_B, theta_dot)
    dx = (r * phi_L_dot / 2) * np.cos(theta) + (r * phi_R_dot / 2) * np.cos(theta)
    dy = (r * phi_L_dot / 2) * np.sin(theta) + (r * phi_R_dot / 2) * np.sin(theta)

    if len(x_goals) > 1 and euc_distance < 0.05:
        x_goals.pop(0)
        y_goals.pop(0)
    return [dx, dy, theta_dot]

# Define initial conditions
y0 = [0, 0, 0]  # x=0, y=0, theta=0

# Define time vector
t = np.linspace(0, 500, 5000)  # Time from 0 to 500 s

# Define desired positions and control parameters
num_points = int(input("Enter the number of desired points: "))
x_goals = []
y_goals = []
for i in range(num_points):
    x_goal = float(input(f"Enter the desired x coordinate for point {i+1}: "))
    y_goal = float(input(f"Enter the desired y coordinate for point {i+1}: "))
    x_goals.append(x_goal)
    y_goals.append(y_goal)

x_goals_copy = x_goals.copy()
y_goals_copy = y_goals.copy()

# Solving the differential equations
sol = odeint(model, y0, t, args=(x_goals, y_goals))
x, y, theta = sol[:, 0], sol[:, 1], sol[:, 2]

plt.close("all")
def plot_trajectory(time):
    index = int(time / (t[-1] / len(t)))
    plt.figure(figsize=(8, 6))
    plt.plot(x[:index], y[:index])
    plt.plot(x_goals_copy, y_goals_copy, 'ro')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Position of the particle in space')
    plt.grid(True)
    plt.show()

time_widget = widgets.FloatSlider(value=0, min=0, max=t[-1], step=0.1, description='Time')
interact(plot_trajectory, time=time_widget);

interactive(children=(FloatSlider(value=0.0, description='Time', max=500.0), Output()), _dom_classes=('widget-…